This notebook has a generic code to extract data from Postgres based on given datalog queries

### IMPORTANT - JOIN Query has not been implemented. Students need to implement this.

In [1]:
import pandas as pd
import psycopg2 as pg
import types

In [2]:
# Connect to an existing database
connection = pg.connect("dbname=SQLBook user=postgres password=Awesome534")

In [3]:
# Datalog queries
datalog = "Ans(numunits, firstname, billdate), orders.orderid > 1000, orders.numunits > 1"
q1 = "orders(numunits, customerid, orderid)"
q2 = "customers(firstname, customerid)"
q3 = "orderlines(billdate, orderid)"

In [4]:
# Given a datalog query of form - "Ans(numunits, firstname, billdate), orders.orderid > 1000, orders.numunits > 1"
# this method extracts the column names from it
def getColumns(datalog):
    return [col.strip() for col in datalog[(datalog.index("(")+1): datalog.index(")")].split(",")]
getColumns(datalog)

['numunits', 'firstname', 'billdate']

In [5]:
# Given a datalog query of form - "Ans(numunits, firstname, billdate), orders.orderid > 1000, orders.numunits > 1"
# this method extracts the where conditions from it
def getWhereConditions(datalog):
    rawString = datalog.split("),")[1]
    conds = [cond.strip() for cond in rawString.split(",")]
    return conds

In [6]:
conds = getWhereConditions(datalog)
print (conds)

['orders.orderid > 1000', 'orders.numunits > 1']


In [7]:
q = [datalog, q1,q2,q3, conds] # Query Processing team to receive this from integrated schema team
q

['Ans(numunits, firstname, billdate), orders.orderid > 1000, orders.numunits > 1',
 'orders(numunits, customerid, orderid)',
 'customers(firstname, customerid)',
 'orderlines(billdate, orderid)',
 ['orders.orderid > 1000', 'orders.numunits > 1']]

In [8]:
# Given a result as a list, this method converts it to the expected format of
# ['Ans(A,B,C)', 'Ans(a,b,c)']
def prettyPrintResult(res):
    result_list = []
    for r in res:
        current = "Ans("
        for ele in r:
            current += str(ele) + ","
        current = current[:-1] 
        current += ")"
        result_list.append(current)
    return result_list

In [9]:
def getWhereConditionForSelectQuery(conds):
    if not conds or len(conds) == 0:
        return
    where_condition = "" + conds[0]
    if len(conds) == 1:
        return where_condition
    for cond in conds[1:]:
        where_condition += " AND " + cond + " "
    return where_condition[:-1]

def getFullyQualifiedColumnNames(q):
    col_names = {}
    for val in q:
        table_name = val[:val.index("(")]
        columns = getColumns(val)
        for col in columns:
            if col in col_names:
                table = col_names.get(col)
                table += ";"+table_name
                col_names[col] = table
            else:
                col_names[col] = table_name
    return col_names

def getQueryColsFromFQCols(column_names, cols_to_return):
    return_str = ""
    for col in cols_to_return:
        if col in column_names:
            table_name = column_names[col]
            if table_name.find(";") == -1:
                return_str += table_name + "." + col + ", "
        else:
            raise Exception('Oh. Something bad happened!')
    return return_str[:-2]

def getTableNames(q):
    return [i[:i.index("(")].strip() for i in q]

def getQueryTableNames(table_names):
    return_str = ""
    for table_name in table_names:
        return_str += table_name + ", "
    return return_str[:-2]

def getJoinConditions(column_names):
    return_str = ""
    for col in column_names:
        if column_names[col].find(";") != -1:
            tables = column_names[col].split(";")
            base_str = tables[0].strip() + "." + col + "="
            for table in tables[1:]:
                return_str += (base_str + table.strip() + "." + col + " AND ")
    return return_str[:-5]

def buildSelectQuery(q):
    datalog = q[0]                               #"Ans(numunits, firstname, billdate), orderid > x, numunits > 1"
    cols_to_return = getColumns(datalog)         #numunits, firstname, billdate
    q = q[1:]                                    #everything except datalog
    
    conds   = [i for i in q if isinstance(i, list)]
    # conds includes the following:
    # [['orders.orderid > 1000', 'orders.numunits > 1']]
    
    selects = [i for i in q if not isinstance(i, list)]
    # selects includes the following:
    # 'orders(numunits, customerid, orderid)',
    #'customers(firstname, customerid)',
    # 'orderlines(billdate, orderid)'
    
    column_names = getFullyQualifiedColumnNames(selects) # Dictionary of {numunits:orders, customerid:orders;customers} etc.
    table_names = getTableNames(selects) # [orders, customers, orderlines]
    
    return "SELECT " + getQueryColsFromFQCols(column_names, cols_to_return) + \
           " FROM " + getQueryTableNames(table_names) + \
           " WHERE " + getJoinConditions(column_names) + \
           " AND " + getWhereConditionForSelectQuery(conds[0]) + " LIMIT 10;"

In [10]:
def getDatalogResult(q):
    cur = connection.cursor()
    select_query = buildSelectQuery(q)
    print (select_query)
    print ()
    cur.execute(select_query)
    res = cur.fetchall()
    return prettyPrintResult(res)

In [11]:
print (getDatalogResult(q))

SELECT orders.numunits, customers.firstname, orderlines.billdate FROM orders, customers, orderlines WHERE orders.customerid=customers.customerid AND orders.orderid=orderlines.orderid AND orders.orderid > 1000 AND orders.numunits > 1 LIMIT 10;

['Ans(2,LAWRENCE,2011-03-08)', 'Ans(2,JAMES,2011-01-20)', 'Ans(4,LINDA,2009-11-12)', 'Ans(4,LINDA,2009-11-20)', 'Ans(2,D.,2009-11-11)', 'Ans(2,D.,2009-11-20)', 'Ans(3,CHARLES,2009-11-10)', 'Ans(2,HEATHER,2010-12-10)', 'Ans(2,HEATHER,2010-12-19)', 'Ans(3,FRANK,2010-12-16)']


## Solr

We need to specify the URL and make a REST call. 

To get results having both iPod and camera, the URL is 
http://localhost:8983/solr/techproducts/select?q=iPod,camera

In [12]:
hostname = "localhost"
port_num = 8983
core_name = "techproducts"
q = ["iPod", "camera"]
query = ",".join(q)
urlToHit = "http://" + hostname + ":" + str(port_num) + "/solr/" + core_name + "/select?q=" + query
print (urlToHit)

http://localhost:8983/solr/techproducts/select?q=iPod,camera


In [13]:
import urllib.request as request

req = request.Request(urlToHit)
result = request.urlopen(req)
resulttext = result.read()
resulttext

URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>